# Direct Collocation Integrator

Using collocation to integrate an ODE system.  Collocation is a high-order implicit integration scheme that works well in practice.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from casadi import MX, SX, DM, Function, Opti
import casadi as cas
from platform import python_version
from numpy.testing import assert_array_equal, assert_array_almost_equal

print(f"Casadi: {cas.__version__}")
print(f"Python: {python_version()}")

Casadi: 3.6.7
Python: 3.10.13


## The Collocation Problem

Consider the case of an autonomous system (no manipulated input)

Solve an initial value problem:

Get to $x(t_0 + \Delta t)$ from $x(t_0)$ given $ \dot{x}(t) = f(t, x)$.

The idea of collocation is to assume a polynomial $\Pi(t)$ as an approximation to $x(t)$ in the interval $(t_0, t_0 + \Delta t)$

Need to find a set of coefficients of the polynomial that best match $ \dot{x}(t) = f(t, x)$.

$$\Pi(t; \text{coefficients})$$

## Exercise

Goal is to integrate the following time-dependent ODE

$$
\frac{d\left[\begin{array}{l}
x_1 \\
x_2
\end{array}\right]}{d t}=\left[\begin{array}{c}
\left(1-x_2^2\right) x_1-x_2+t \\
x_1
\end{array}\right]
$$

from $t_0 = 2$ to $t_f = 2.1$, with an initial value of $x_0 = \left[ \begin{array}{c} 1 \\ 0.5 \end{array} \right]$.

## 1. Implement CasADi Function of Righthand Side of ODE

Implement a CasADi Function $f$ that maps from time and state to the ODE’s right-hand side.

Create suitable MX symbols and use the following boilerplate:

```python
f = Function('f', list_of_input_symbols, list_of_output_expressions,
             list_of_input_labels, list_of_output_labels)
```

In [2]:
t = MX.sym('t')
x = MX.sym('x', 2)
x1, x2 = x[0], x[1]

rhs = cas.vertcat(
    (1 - x2 ** 2) * x1 - x2 + t,
    x1
)
f = Function('f', [t, x], [rhs], ['t', 'x'], ['dxdt'])
f

Function(f:(t,x[2])->(dxdt[2]) MXFunction)

In [3]:
# Test
t0 = 2
dt = 0.1
tf = t0 + dt
x0 = cas.vertcat(1, 0.5)

assert_array_equal(np.array(f(t0, x0)).reshape(-1), [2.25, 1])